# Train the YOLO Model to the dataset

In [1]:
import torch
import sys

print(f"Python Version: {sys.version}")
print(f"Torch Version: {torch.__version__}")

# 1. Prüfen ob CUDA generell verfügbar ist
if not torch.cuda.is_available():
    print(" KEINE GPU GEFUNDEN! Torch läuft auf CPU.")
else:
    try:
        # 2. Treiber-Status abfragen (System-Level)
        print("\n--- System GPU Status (nvidia-smi) ---")
        !nvidia-smi
        
        # 3. Harter Test: Eine Tensor-Berechnung auf der GPU durchführen
        # Das entlarvt den 'Suspend-Bug', bei dem is_available() True sagt, aber nichts geht.
        x = torch.tensor([1.0, 2.0]).cuda()
        print(f"\n GPU TEST ERFOLGREICH: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        
    except Exception as e:
        print(f"\n GPU FEHLER: Der Treiber scheint zu hängen (Suspend Bug?).")
        print(f"Fehler: {e}")
        print("Lösung: PC Neustarten.")

Python Version: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
Torch Version: 2.9.1+cu128

--- System GPU Status (nvidia-smi) ---
Mon Dec 22 17:57:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:2D:00.0  On |                  N/A |
| 44%   51C    P8             43W /  350W |     939MiB /  24576MiB |     36%      Default |
|      

In [2]:
from ultralytics import YOLO

# Wähle das Modell:
# 'yolo11n.pt' -> Nano (Schnellster Test)
# 'yolo11m.pt' -> Medium (Gute Balance für später)
# 'yolo11l.pt' -> Large (Hohe Genauigkeit für 3090)
MODEL_VARIANT = 'yolo11n.pt' 

print(f"Lade Modell: {MODEL_VARIANT}...")
model = YOLO(MODEL_VARIANT)

# Training starten
results = model.train(
    data='../../dataset/04_model_ready/data.yaml', # Pfad muss stimmen!
    epochs=50,          # Für Nano Test reichen 50. Für Large später 100+
    imgsz=640,          # Standard. Bei 3090 könntest du auch 1280 testen (High Res)
    batch=32,           # 3090 packt bei Nano locker 64/128, bei Large ca 16/24
    name='compound_yolo11n_sim2real',
    device=0,           # Erzwingt GPU 0
    workers=8,          # Schnelleres Laden der Daten
    exist_ok=True       # Überschreibt den Ordner falls er existiert (gut zum Testen)
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/tobiasponeschtu/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Lade Modell: yolo11n.pt...
Ultralytics 8.3.240 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24112MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../dataset/04_model_ready/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015

## Second YOLO run on the new, stratified and oversampled Dataset

In [ ]:
from ultralytics import YOLO

# --- KONFIGURATION ---
# Für den ersten schnellen Pipeline-Check ist Nano super.
# Sobald das läuft -> Auf 'yolo11m.pt' oder 'yolo11l.pt' wechseln!
MODEL_VARIANT = 'yolo11n.pt' 

print(f"Lade Modell: {MODEL_VARIANT}...")
model = YOLO(MODEL_VARIANT)

# Training starten
results = model.train(
    data='../../dataset/04_model_ready/data.yaml', 
    epochs=50,          
    
    # TIPP FÜR SCIENTIFIC FIGURES:
    # 640px ist oft zu klein für feine Achsenbeschriftungen.
    # Da du eine 3090 hast, probier direkt 1024 oder 1280, 
    # das hilft massiv bei kleinen Subpanels!
    imgsz=640,          # Für den Nano-Schnelltest ok. Später: 1280
    
    # TIPP FÜR RTX 3090:
    # Bei Nano kannst du hier locker 128 oder 256 nehmen.
    # Bei Medium/Large sind 16-32 realistisch.
    batch=64,           
    
    name='compound_yolo11n_sim2real_v1',
    device=0,           
    workers=8,
    
    # WICHTIG: Plots werden gespeichert, damit wir die Confusion Matrix sehen
    plots=True,
    
    # Caching im RAM beschleunigt das Training massiv, wenn du genug RAM (System) hast
    cache=True          
)

Lade Modell: yolo11n.pt...
New https://pypi.org/project/ultralytics/8.3.241 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.240 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3090, 24112MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../dataset/04_model_ready/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=F